In [1]:
import pandas as pd
import os

export_path = r"C:\Users\User\Downloads\COB DATA\2025\10.October\10.25.25\Export"
import_path = r"C:\Users\User\Downloads\COB DATA\2025\10.October\10.25.25\COB\Data"
os.makedirs(export_path, exist_ok=True)


In [2]:
#Price List M0

im_filename_pl = "DMS-Price-on.xlsx"
full_im_path_pl = os.path.join(import_path, im_filename_pl)

pl_df = pd.read_excel(full_im_path_pl)

pl_df2 = pl_df

pl_df2 = pl_df2[['product_code', 'product_description', 'uom_description', 'selling_price', 'cust_class', 'cust_channel']]
pl_df2.insert(4,'with vat', 0)
pl_df2['with vat'] = pl_df2['selling_price'] * 1.12

pl_df_m0 = pl_df2

pl_df_m0 = pl_df_m0[pl_df_m0['cust_class'] != 'BEV Dealer']
pl_df_m0 = pl_df_m0[pl_df_m0['cust_channel'] != 'VAN(EXTRUCK)']

pl_df_m0_f = pl_df_m0.pivot_table(index=['product_code', 'product_description',], columns='uom_description', values='with vat', aggfunc=sum)

pl_df_m0_f_1 = pl_df_m0_f.reset_index()

pl_df_m0_f_1['Case'] = pl_df_m0_f_1['Case'].fillna(0)
pl_df_m0_f_1['Piece'] = pl_df_m0_f_1['Piece'].fillna(0)
pl_df_m0_f_1['Subcase'] = pl_df_m0_f_1['Subcase'].fillna(0)

pl_m0_final = pl_df_m0_f_1[['product_code', 'product_description', 'Case', 'Subcase', 'Piece']]

ex_filename_pl_m0 = "M0 Pricelist.xlsx"
full_path_pl_m0 = os.path.join(export_path, ex_filename_pl_m0)

export_pl_m0 = pl_m0_final
export_pl_m0.to_excel(full_path_pl_m0, index=False)


c:\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\User\AppData\Local\Temp\ipykernel_13440\2121799534.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pl_df2['with vat'] = pl_df2['selling_price'] * 1.12
C:\Users\User\AppData\Local\Temp\ipykernel_13440\2121799534.py:19: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pl_df_m0_f = pl_df_m0.pivot_table(index=['product_code', 'product_description